In [ ]:
# Define your parameters

WIND_DIRECTION = 'south_west' # set an area destination

# set percent od every area you need
landuse_percent = {
    'living': 0.543,
    'social': 0.066,
    'industrial': 0.061,
    'transport': 0.077,
    'recreation': 0.253
}